<a href="https://colab.research.google.com/github/erjavaid/ensemble-technique/blob/main/Bgging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Random forest classifier pipelining and hyperparameters**

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [7]:
# load dataset
df=sns.load_dataset('tips')

In [8]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [10]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [11]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [12]:
df.isnull().sum()

,0
total_bill,0
tip,0
sex,0
smoker,0
day,0
time,0
size,0


In [13]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [14]:
# convert time inri 0 and 1 using label encoding
df['time']=df['time'].map({'Lunch':0,'Dinner':1})

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
label=LabelEncoder()

In [17]:
df['time']=label.fit_transform(df['time'])

In [18]:
df['time'].unique()

array([1, 0])

In [19]:
# dependent nd independent features
X=df.drop('time',axis=1)
y=df['time']

In [20]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [21]:
y

,time
0,1
1,1
2,1
3,1
4,1
...,...
239,1
240,1
241,1
242,1


In [22]:
# split train and test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [23]:
X_train

,total_bill,tip,sex,smoker,day,size
228,13.28,2.72,Male,No,Sat,2
208,24.27,2.03,Male,Yes,Sat,2
96,27.28,4.00,Male,Yes,Fri,2
167,31.71,4.50,Male,No,Sun,4
84,15.98,2.03,Male,No,Thur,2
...,...,...,...,...,...,...
106,20.49,4.06,Male,Yes,Sat,2
14,14.83,3.02,Female,No,Sun,2
92,5.75,1.00,Female,Yes,Fri,2
179,34.63,3.55,Male,Yes,Sun,2


In [24]:
y_test

,time
24,1
6,1
153,1
211,1
198,0
176,1
192,0
124,0
9,1
101,1


In [25]:
y_train.shape

(195,)

In [30]:
# use pipelining automation
from sklearn.compose import ColumnTransformer # comining the pipes into a single pipe
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer # Hnadling Missing values
from sklearn.preprocessing import OneHotEncoder # convert categorical into numerical
from sklearn.preprocessing import StandardScaler # for feature scaling


In [31]:
categorical_cols=['sex','smoker','day']
numerical_cols=['total_bill','tip','size']

In [32]:
# automation
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('onehotencoder',OneHotEncoder())
    ]
)

In [33]:
num_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [34]:
cat_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder', OneHotEncoder())])

In [35]:
# now compine these two pipelines
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [36]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day'])])

In [37]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [38]:
X_train

array([[-0.79306155, -0.2580329 , -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.46322744, -0.74211442, -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.80730659,  0.6399734 , -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.65383098, -1.46472887, -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.64749986,  0.32426806, -0.61214068, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.75289699, -0.41237773,  0.45363997, ...,  1.        ,
         0.        ,  0.        ]])

In [39]:
X_test

array([[-0.04546101,  0.06468811, -0.61214068,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-1.30860871, -0.76316144, -0.61214068,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.4952348 , -0.76316144,  1.51942062,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.64841289,  1.45379161,  1.51942062,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.82506891, -0.76316144, -0.61214068,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.26608319, -0.76316144, -0.61214068,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.   

In [40]:
# model building using raandom forest
from sklearn.ensemble import RandomForestClassifier


In [42]:
rfc=RandomForestClassifier()

In [43]:
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [44]:
# prediction
y_pred=rfc.predict(X_test)

In [45]:
# calculate the accuracy
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [46]:
score=accuracy_score(y_test,y_pred)

In [47]:
score

0.9591836734693877

In [48]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94        15
           1       1.00      0.94      0.97        34

    accuracy                           0.96        49
   macro avg       0.94      0.97      0.95        49
weighted avg       0.96      0.96      0.96        49



In [50]:
# hyperparametre tuning
from sklearn.model_selection import RandomizedSearchCV

In [52]:
param={
    'n_estimators':[10,20,30,40,50,60,70,80,90,100],
    'criterion':['gini','entropy'],
    'max_depth':[2,4,6,8,10,None],
    'min_samples_split':[2,4,6,8,10],




}

In [54]:
rsc=RandomizedSearchCV(estimator=rfc,param_distributions=param,cv=4,verbose=2,n_jobs=1,scoring='accuracy')

In [55]:
rsc.fit(X_train,y_train)

Fitting 4 folds for each of 10 candidates, totalling 40 fits
[CV] END criterion=gini, max_depth=None, min_samples_split=6, n_estimators=10; total time=   0.1s
[CV] END criterion=gini, max_depth=None, min_samples_split=6, n_estimators=10; total time=   0.1s
[CV] END criterion=gini, max_depth=None, min_samples_split=6, n_estimators=10; total time=   0.1s
[CV] END criterion=gini, max_depth=None, min_samples_split=6, n_estimators=10; total time=   0.0s
[CV] END criterion=gini, max_depth=8, min_samples_split=8, n_estimators=80; total time=   0.3s
[CV] END criterion=gini, max_depth=8, min_samples_split=8, n_estimators=80; total time=   0.5s
[CV] END criterion=gini, max_depth=8, min_samples_split=8, n_estimators=80; total time=   0.2s
[CV] END criterion=gini, max_depth=8, min_samples_split=8, n_estimators=80; total time=   0.2s
[CV] END criterion=gini, max_depth=10, min_samples_split=8, n_estimators=40; total time=   0.1s
[CV] END criterion=gini, max_depth=10, min_samples_split=8, n_estimator

RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(), n_jobs=1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [2, 4, 6, 8, 10, None],
                                        'min_samples_split': [2, 4, 6, 8, 10],
                                        'n_estimators': [10, 20, 30, 40, 50, 60,
                                                         70, 80, 90, 100]},
                   scoring='accuracy', verbose=2)

In [56]:
rsc.best_params_

{'n_estimators': 40,
 'min_samples_split': 6,
 'max_depth': 2,
 'criterion': 'gini'}

In [57]:
y_pre=rsc.predict(X_test)

In [58]:
accuracy_score(y_test,y_pre)

1.0

In [59]:
# Bagging
from sklearn.ensemble import BaggingClassifier

In [60]:
# model training Automation
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [61]:
models={
    'RandomForestClassifier':RandomForestClassifier(),
    'LogisticRegression':LogisticRegression(),
    'DecisionTreeClassifier':DecisionTreeClassifier(),
    'KNeighborsClassifier':KNeighborsClassifier(),
    'SVC':SVC()
}


In [62]:
from sklearn.metrics import accuracy_score


In [63]:
# create user defined function
def evaluate_model(X_train,X_test,y_train,y_test,models):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        model.fit(X_train,y_train)
        y_pred=model.predict(X_test)
        accuracy=accuracy_score(y_test,y_pred)
        report[list(models.keys())[i]]=accuracy
    return report

In [64]:
# calling function
evaluate_model(X_train,X_test,y_train,y_test,models)

{'RandomForestClassifier': 0.9591836734693877,
 'LogisticRegression': 1.0,
 'DecisionTreeClassifier': 0.9387755102040817,
 'KNeighborsClassifier': 0.9183673469387755,
 'SVC': 0.9591836734693877}